In [1]:
from typing import *
from collections import namedtuple
from itertools import groupby
import re

In [2]:
RomanToDecimalTest = namedtuple('RomanToDecimalTest', 'roman_number number')

def test_roman_to_decimal():
    for test_value in TEST_VALUES:
        result = RomanToDecimal(test_value.roman_number)
        assert result.result == test_value.number, f'{result.result=}, {test_value.number=}'

def test_decimal_to_roman():
    for test_value in TEST_VALUES:
        result = DecimalToRoman(test_value.number)
        assert result.result == test_value.roman_number, f'{result.result=}, {test_value.roman_number=}'

def test_roman_to_decimal_2():
    for test_value in TEST_VALUES:
        result = RomanToDecimal2(test_value.roman_number)
        assert result.result == test_value.number, f'{result.result=}, {test_value.number=}'

TEST_VALUES = [
    RomanToDecimalTest('I', 1),
    RomanToDecimalTest('II', 2),
    RomanToDecimalTest('III', 3),
    RomanToDecimalTest('IV', 4),
    RomanToDecimalTest('V', 5),
    RomanToDecimalTest('VI', 6),
    RomanToDecimalTest('IX', 9),
    RomanToDecimalTest('X', 10),
    RomanToDecimalTest('XII', 12),
    RomanToDecimalTest('XV', 15),
    RomanToDecimalTest('XXIV', 24),
    RomanToDecimalTest('XXV', 25),
    RomanToDecimalTest('XXXVII', 37),
    RomanToDecimalTest('CIV', 104),
    RomanToDecimalTest('M', 1000),
    RomanToDecimalTest('MM', 2000),
    RomanToDecimalTest('MMMCMXCIX', 3999),
]

In [3]:
class NotRomanNumberException(Exception):
    pass

In [4]:
def _is_roman(string: str, letters: List[str]) -> bool:
    izi_regex = re.compile(r'^(?=[MDCLXVI])M*(C[MD]|D?C{0,3})(X[CL]|L?X{0,3})(I[XV]|V?I{0,3})$')
    groups = groupby(string)
    # All letters are romman letters
    if not all([letter in letters for letter in string]):
        return False
    # If any of the romman letters is repeated more than 3 times in a row, is not roman number
    if any([count > 3 for _, count in [(label, sum(1 for _ in group)) for label, group in groups]]):
        return False
    # If any of the two letter romman numbers is repeated more than once, is not roman number
    if any([len(string.split(double_letter)) - 1 > 1 for double_letter in filter(lambda x: len(x) == 2, letters)]):
        return False
    if not izi_regex.match(string.upper()):
        return False
    return True

In [5]:

class RomanToDecimal:
    letters: Dict[str, int] = dict(i=1, v=5, x=10, l=50, c=100, d=500, m=1000)
    total: int
    roman_number: str
    result: Optional[int] = None

    def __init__(self, roman_number: str) -> None:
        if not isinstance(roman_number, str):
            raise NotRomanNumberException(f'{roman_number=} must be a "str" instance')
        if not _is_roman(roman_number.lower(), self.letters.keys()):
            raise NotRomanNumberException(f'{roman_number=} is not a valid roman number')

        self.roman_number = roman_number.lower()
        self.number_list = [self.letters[letter] for letter in self.roman_number]
        self()

    def __repr__(self) -> str:
        return f'{self.roman_number.upper()} = {self.result}'

    def __getitem__(self, index) -> Optional[int]:
        try:
            return self.number_list[index]
        except IndexError:
            return None

    def __iter__(self) -> Iterator[Tuple[int, Optional[int]]]:
        for i, curr in enumerate(self.number_list):
            yield curr, self[i+1] 

    def __call__(self):
        self.result = sum([curr if next is None or curr >= next else -curr for curr, next in self])

In [6]:
RomanToDecimal('XXIV')

XXIV = 24

In [7]:
class DecimalToRoman:
    number: int
    roman_number: str = ''
    numbers: Dict[int, str] = {
        1: 'i', 4: 'iv', 5: 'v', 9: 'ix', 10: 'x', 40: 'xl',
        50: 'l', 90: 'xc', 100: 'c', 400: 'cd', 500: 'd', 900: 'cm', 1000: 'm'
    }
    result: Optional[str] = None

    def __init__(self, number: int) -> None:
        if not isinstance(number, int):
            raise NotRomanNumberException(f'{number=} must be an "int" instance')
        if number > 3999:
            raise NotRomanNumberException(f'{number=} is greater than max representation of roman number "3999"')
        self.number = number
        self()
        
    def __repr__(self) -> str:
        return f'{self.number} = {self.result}'

    def __iter__(self) -> Iterator[Tuple[int, str]]:
        for k, v in reversed(self.numbers.items()):
            yield k, v

    def __call__(self):
        total = self.number
        self.result = ''
        while total > 0:
            for number, letter in self:
                if total / number >= 1:
                    total -= number
                    self.result += letter.upper()
                    break

In [8]:
DecimalToRoman(3999)

3999 = MMMCMXCIX

In [9]:
class RomanNumber:
    def __init__(self, roman_number: str) -> None:
        self.roman_number = roman_number.lower()

    def __str__(self) -> str:
        return self.roman_number.upper()

    def __len__(self) -> int:
        return len(self.roman_number)
    
    def __iter__(self) -> Iterator[Tuple[Optional[str], str, Optional[str]]]:
        for i, curr in enumerate(self.roman_number):
            yield self[i - 1], curr, self[i + 1]
        
    def __getitem__(self, index) -> Optional[str]:
        if index < 0: return None
        try:
            return self.roman_number[index]
        except IndexError:
            return None

class RomanToDecimal2:
    letters: Dict[str, int] = dict(i=1, iv=4, v=5, ix=9, x=10, xl=40, l=50, xc=90, c=100, cd=400, d=500, cm=900, m=1000)
    result: Optional[int] = None

    def __init__(self, roman_number: str) -> None:
        if not isinstance(roman_number, str):
            raise NotRomanNumberException(f'{roman_number=} must be a "str" instance')
        if not _is_roman(roman_number.lower(), self.letters.keys()):
            raise NotRomanNumberException(f'{roman_number=} is not a valid roman number')
        self.roman_number = RomanNumber(roman_number)
        self()

    def __repr__(self) -> str:
        return str(self)

    def __str__(self) -> str:
        if not hasattr(self, 'result'): self()
        return f'{self.roman_number} = {self.result}'

    def __len__(self) -> int:
        return len(self.roman_number)

    def __getitem__(self, index) -> int:
        return self.letters[index]
        
    def __contains__(self, key) -> bool:
        return key in self.letters

    def __call__(self) -> int:
        string = ''
        for previous_letter, letter, next_letter in self.roman_number:
            if letter and next_letter and letter + next_letter in self:
                continue
            if previous_letter and letter and (rsl := previous_letter + letter) in self:
                string += '*' * self[rsl]
                continue
            string += '*' * self[letter]
        self.result = len(string)
        return self.result

a = RomanToDecimal2('IV')
a

IV = 4

In [10]:
test_roman_to_decimal()

In [11]:
test_roman_to_decimal_2()